In [1]:
import os
import re
import time
import requests

WIKIPEDIA_API_URL_EN = "https://en.wikipedia.org/w/api.php"

def sanitize_filename(filename: str) -> str:
    """
    Reemplaza caracteres no válidos en el nombre de archivo por '_'.
    """
    return re.sub(r'[\\/*?:"<>|]', '_', filename)

def get_category_members(category: str, cmcontinue: str = None) -> dict:
    """
    Devuelve los miembros de una categoría (artículos, subcategorías) utilizando la API de Wikipedia.
    Puede devolver un 'continue' token para seguir paginando resultados.
    """
    params = {
        "action": "query",
        "format": "json",
        "list": "categorymembers",
        "cmtitle": category,
        "cmlimit": "max",       # recupera el máximo número de resultados permitido
    }
    if cmcontinue:
        params["cmcontinue"] = cmcontinue

    response = requests.get(WIKIPEDIA_API_URL_EN, params=params)
    data = response.json()
    return data

def get_page_content(page_id: int) -> str:
    """
    Utiliza la API de Wikipedia para obtener el contenido 'wikitext' de una página por su ID.
    (También se podría usar 'prop=extracts' con 'explaintext=true' para recibir el texto plano).
    """
    params = {
        "action": "query",
        "prop": "revisions",
        "rvprop": "content",
        "rvslots": "main",
        "format": "json",
        "pageids": page_id,
    }
    response = requests.get(WIKIPEDIA_API_URL_EN, params=params)
    data = response.json()
    pages = data.get("query", {}).get("pages", {})
    if not pages:
        return ""
    page_data = pages.get(str(page_id), {})
    if "revisions" not in page_data:
        return ""
    # El contenido está en 'slots' -> 'main' -> '*'
    wikitext = page_data["revisions"][0]["slots"]["main"].get("*", "")
    return wikitext

def download_wikipedia_articles(category: str,
                                output_dir: str,
                                visited_categories=None):
    if visited_categories is None:
        visited_categories = set()

    if category in visited_categories:
        return
    visited_categories.add(category)

    cmcontinue = None

    while True:
        data = get_category_members(category, cmcontinue)
        if "error" in data:
            print(f"Ocurrió un error: {data['error']}")
            break

        category_members = data.get("query", {}).get("categorymembers", [])
        for member in category_members:
            page_title = member["title"]
            page_id = member["pageid"]
            
            # NS = 14 → subcategoría, NS = 0 → artículo
            ns = member["ns"]
            
            if ns == 14:
                # Es una subcategoría
                subcat = "Category:" + page_title.replace("Category:", "")
                download_wikipedia_articles(subcat, output_dir, visited_categories)

            elif ns == 0:
                # Es un artículo normal
                print(f"Descargando artículo: {page_title}")
                content = get_page_content(page_id)
                if content:
                    safe_title = sanitize_filename(page_title)
                    file_path = os.path.join(output_dir, f"{safe_title}.txt")
                    with open(file_path, "w", encoding="utf-8") as f:
                        f.write(content)
                # Pequeña pausa para evitar saturar la API
                time.sleep(0.2)

        # Paginación
        if "continue" in data:
            cmcontinue = data["continue"].get("cmcontinue")
            if not cmcontinue:
                break
        else:
            break



In [ ]:

def main():
    # Nombre de la categoría principal sobre WWII en inglés:
    # Wikipedia usa "Category:World War II"
    # Puedes ajustar si te interesa una subcategoría concreta
    #cat = input("Indica la categoría a descargar")
    category = "World_War_II"
    main_category = f"Category:{category}"

    # Carpeta de salida: Data/Wikipedia/English
    base_dir = f"Data/{category}/Wikipedia/English"
    os.makedirs(base_dir, exist_ok=True)

    print(f"Descargando artículos de la categoría '{main_category}' en '{base_dir}'...")
    download_wikipedia_articles(main_category, base_dir)
    print("Proceso completado.")

if __name__ == "__main__":
    main()


Descargando artículos de la categoría 'Category:World_War_II' en 'Data/World_War_II/Wikipedia/English'...
Descargando artículo: World War II
Descargando artículo: Outline of World War II
Descargando artículo: 1945 Resko Przymorskie Dornier Do 24 crash
Descargando artículo: Attack on the battleship Tirpitz by submarine K-21
Descargando artículo: Axis powers
Descargando artículo: Battle of Hucisko
Descargando artículo: Battle of Jaktorów
Descargando artículo: Battle of Lạng Sơn (1940)
Descargando artículo: ORP Czapla
Descargando artículo: France Forever
Descargando artículo: Gustav Klimt
Descargando artículo: USCGC Northland (WPG-49)
Descargando artículo: Open city
Descargando artículo: Peace efforts during World War II
Descargando artículo: Royal Navy during the Second World War
Descargando artículo: Student soldier
Descargando artículo: World War II combatives
Descargando artículo: Africa Service Medal
Descargando artículo: British military post offices in Africa
Descargando artículo: 